# **EXPERIMENTS**

## **EXPERIMENT 2 OCR VS Visual Language Model**

### TEST-1: Extraction with LLAVA

In [1]:
import base64
import requests
import os

# List of image file paths
image_paths = [
    "0/png/5a6b122c-39c1-4581-8c1f-2d6f36a9f8a0_4.png", 
    "0/png/fe245192-1f9c-4f28-9b32-046fb7ce7e1e_13.png", 
    "0/png/9fe6b262-9944-417d-a0c4-9f2de1de2994_4.png", 
    "0/png/12756724-394c-4576-b373-7c53f1abbd94_0.png", 
    "0/png/1a20ded1-50fc-4153-9a95-e158eeb7199e.png", 
    "0/png/eda5b003-9250-4913-b724-74cca86240af_14.png", 
    "0/png/64bba692-d430-440c-9f1e-2575f45770af_1.png", 
    "0/png/912204cb-8ab7-48b8-9abf-d803f3804d08_7.png", 
    "0/png/12756724-394c-4576-b373-7c53f1abbd94_5.png", 
    "0/png/b8cea3b1-4dde-4438-9b1a-6faf690bbad0.png"
]

# Define the prompt
prompt = "EXTRACT AND ONLY GIVE ME THE Chinese text in the given image. NO TRANSLATION, ONLY extract the TEXT IN CHINESE MANDARIN FROM THE PNG FILE"

# Process each image
for image_path in image_paths:
    with open(image_path, "rb") as f:
        image_b64 = base64.b64encode(f.read()).decode("utf-8")

    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "llava:13b",
            "prompt": prompt,
            "images": [image_b64],
            "stream": False
        },
    )

    filename = os.path.basename(image_path)
    if response.ok:
        output_text = response.json()["response"]
    else:
        output_text = f"Error: {response.text}"

    print(f"{filename}:\n{output_text.strip()}\n")

5a6b122c-39c1-4581-8c1f-2d6f36a9f8a0_4.png:
这是一段中文（汉语）的文本，我无法直接为您提供图像。不过，如果您能够提供一个描述或提供一下您想要了解的内容，我可以帮助您理解这段文本或者回答与中文相关的问题。

fe245192-1f9c-4f28-9b32-046fb7ce7e1e_13.png:
The text in the image is as follows:
```
1. 系统测试功能的介绍。
2. 前往详细操作指南中的链接页面。
3. 如果需要更多关于这个功能的信息，请访问官方网站或联系我们的客户服务部门。
```

9fe6b262-9944-417d-a0c4-9f2de1de2994_4.png:
很抱歉，我不能提供文字的识别或翻译服务。请使用其他资源来获得所需的信息。

12756724-394c-4576-b373-7c53f1abbd94_0.png:
The Chinese text in the image is:

赵玉华教授领导下，我国中医药研究的先进性和实用性得到了显著提升。感谢全国中医师同仁们一直以来对我这个工作的支持和关心，让我能在这里成为一名真正的中医研究人才。在未来，中医护理将继续深入发展，不断提高自身的技术水平和服务能力。

1a20ded1-50fc-4153-9a95-e158eeb7199e.png:
The image contains a screenshot of a mobile phone with a messaging app displaying a conversation. Here is the Chinese text extracted from the image:

```
你好呀？我想问你一下。是不是疲了？
你懂我在说什么吗？
```

eda5b003-9250-4913-b724-74cca86240af_14.png:
很抱歉，我无法提供文本提取服务。

64bba692-d430-440c-9f1e-2575f45770af_1.png:
I'm sorry, but I'm unable to provide any assistance with recognizing or transcribing characters or t

## **EXPERIMENT 1 DATA TRANSLATION - 3 LLMs**

### TEST-1: gemma3:27b

In [79]:
import pandas as pd
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

csv_path = 'csvs/10.csv'
output_path = '3llms_10.csv'  # Output in project root directory
df = pd.read_csv(csv_path)

def get_context(df, index, window=5):
    start = max(0, index - window)
    end = min(len(df), index + window + 1)
    return "\n".join(df['Message'].iloc[start:end].dropna())

def query_ollama(prompt, model="gemma3:27b"):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }
    try:
        response = requests.post(url, json=payload, timeout=90)
        response.raise_for_status()
        return response.json()["response"].strip()
    except Exception as e:
        print(f"Ollama error: {e}")
        return "[Translation Error]"

def translate_row(index):
    target = df.at[index, 'Message']
    if pd.isna(target):
        return index, ""
    context = get_context(df, index)
    prompt = (
        "You are translating Chinese messages to English. Below is a series of related messages. "
        "Use the full context to understand the meaning, but only translate the specific message provided.\n\n"
        f"Context:\n{context}\n\n"
        f"Message to translate:\n{target}\n\n"
        "ONLY RETURN the English translation of the message, ANYTHING ELSE IS FORBIDDEN! "
        "If you encounter a filename or file reference, preserve it exactly as it appears, DON'T add anything!"
    )
    translation = query_ollama(prompt)
    return index, translation

translations = [""] * len(df)

futures = []
with ThreadPoolExecutor(max_workers=8) as executor:
    for index in df.index:
        futures.append(executor.submit(translate_row, index))

    for future in tqdm(as_completed(futures), total=len(futures), desc="Translating with Ollama"):
        idx, result = future.result()
        translations[idx] = result

translated_df = pd.DataFrame({'gemma3:27b': translations})
translated_df.to_csv(output_path, index=False)

Translating with Ollama:   7%|▋         | 22/329 [02:06<29:27,  5.76s/it] 


KeyboardInterrupt: 

Ollama error: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=90)
Ollama error: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=90)
Ollama error: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=90)
Ollama error: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=90)
Ollama error: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=90)Ollama error: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=90)
Ollama error: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=90)

### TEST-2: 7shi/llama-translate:8b-q4_K_M

In [10]:
import pandas as pd
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

source_csv_path = 'csvs/10.csv'
existing_translation_path = '3llms_10.csv'
new_model_name = '7shi/llama-translate:8b-q4_K_M'

# Load original messages and previously translated file
source_df = pd.read_csv(source_csv_path)
translated_df = pd.read_csv(existing_translation_path)

def get_context(df, index, window=5):
    start = max(0, index - window)
    end = min(len(df), index + window + 1)
    return "\n".join(df['Message'].iloc[start:end].dropna())

def query_ollama(prompt, model):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }
    try:
        response = requests.post(url, json=payload, timeout=90)
        response.raise_for_status()
        return response.json()["response"].strip()
    except Exception as e:
        print(f"Ollama error: {e}")
        return "[Translation Error]"

def translate_row(index):
    target = source_df.at[index, 'Message']
    if pd.isna(target):
        return index, ""
    context = get_context(source_df, index)
    prompt = (
        "Transalte Mandarin to English. Below is a series of related messages. "
        f"Context:\n{context}\n\n"
        f"Message to translate:\n{target}\n\n"
    )
    translation = query_ollama(prompt, model=new_model_name)
    return index, translation

# Prepare for parallel processing
translations = [""] * len(source_df)
futures = []
with ThreadPoolExecutor(max_workers=8) as executor:
    for index in source_df.index:
        futures.append(executor.submit(translate_row, index))

    for future in tqdm(as_completed(futures), total=len(futures), desc=f"Translating with {new_model_name}"):
        idx, result = future.result()
        translations[idx] = result

translated_df[new_model_name] = translations
translated_df.to_csv(existing_translation_path, index=False)

Translating with 7shi/llama-translate:8b-q4_K_M: 100%|██████████| 149/149 [08:52<00:00,  3.57s/it]


### TEST-3: yi:34b

In [11]:
import pandas as pd
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# File paths
source_csv_path = 'csvs/10.csv'
existing_translation_path = '3llms_10.csv'
new_model_name = 'yi:34b'

# Load original source messages and existing translations
source_df = pd.read_csv(source_csv_path)
translated_df = pd.read_csv(existing_translation_path)

# Context window function
def get_context(df, index, window=5):
    start = max(0, index - window)
    end = min(len(df), index + window + 1)
    return "\n".join(df['Message'].iloc[start:end].dropna())

def query_ollama(prompt, model):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }
    try:
        response = requests.post(url, json=payload, timeout=90)
        response.raise_for_status()
        return response.json()["response"].strip()
    except Exception as e:
        print(f"Ollama error: {e}")
        return "[Translation Error]"

# Translate a single row
def translate_row(index):
    target = source_df.at[index, 'Message']
    if pd.isna(target):
        return index, ""
    context = get_context(source_df, index)
    prompt = (
        "You are translating Chinese messages to English. Below is a series of related messages. "
        "Use the full context to understand the meaning, but only translate the specific message provided.\n\n"
        f"Context:\n{context}\n\n"
        f"Message to translate:\n{target}\n\n"
        "ONLY RETURN the English translation of the message, ANYTHING ELSE IS FORBIDDEN! "
        "If you encounter a filename or file reference, preserve it exactly as it appears, DON'T add anything!"
    )
    translation = query_ollama(prompt, model=new_model_name)
    return index, translation

# Run translations in parallel
translations = [""] * len(source_df)
futures = []
with ThreadPoolExecutor(max_workers=8) as executor:
    for index in source_df.index:
        futures.append(executor.submit(translate_row, index))

    for future in tqdm(as_completed(futures), total=len(futures), desc=f"Translating with {new_model_name}"):
        idx, result = future.result()
        translations[idx] = result

translated_df[new_model_name] = translations
translated_df.to_csv(existing_translation_path, index=False)

Translating with yi:34b: 100%|██████████| 149/149 [07:20<00:00,  2.96s/it]


### Levenshtein Distance Between the 3 Translations

In [14]:
import pandas as pd
import Levenshtein
from itertools import combinations

csv_path = '3llms_10.csv'
df = pd.read_csv(csv_path)

# Identify all translation columns 
translation_columns = [col for col in df.columns if not col.startswith('Levenshtein')]

# Compute Levenshtein distance for all possible combinations
for col1, col2 in combinations(translation_columns, 2):
    new_col_name = f"Levenshtein ({col1} vs {col2})"
    df[new_col_name] = [
        Levenshtein.distance(str(a), str(b)) if pd.notna(a) and pd.notna(b) else None
        for a, b in zip(df[col1], df[col2])
    ]

# Save updated DataFrame back to the same CSV
df.to_csv(csv_path, index=False)

# TEST IMG CLASSIFICATION for all file types - 100% accuracy

In [8]:
import base64
import requests
import os
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# List of image file paths
image_paths = [
    # Financial Info Here (Employee Salary)
    "0/png/3348953d-66e9-4cac-8675-65bb5f2ef929_1_0.png",
    "0/png/3348953d-66e9-4cac-8675-65bb5f2ef929_2_0.png",
    "0/png/3348953d-66e9-4cac-8675-65bb5f2ef929_2_1.png",
    "0/png/3348953d-66e9-4cac-8675-65bb5f2ef929_3_0.png",
    "0/png/3348953d-66e9-4cac-8675-65bb5f2ef929_4_0.png",
    "0/png/3348953d-66e9-4cac-8675-65bb5f2ef929_5_0.png",

    # Product Documentation
    "0/png/12756724-394c-4576-b373-7c53f1abbd94_39.png",
    "0/png/12756724-394c-4576-b373-7c53f1abbd94_47.png",
    "0/png/12756724-394c-4576-b373-7c53f1abbd94_9.png",
    "0/png/12756724-394c-4576-b373-7c53f1abbd94_49.png",
    "0/png/585875ff-f8c5-4a02-acd7-fef37dc9ff11_5.png",

    # Chats
    "0/png/645dfc97-3268-4e1d-920d-4138545456fa.png",
    "0/png/6e9aced1-df28-4e57-b7c8-641609ff4450.png",

    # Employee Information
    "0/png/6d7fc7b3-c892-4cb5-bd4b-a5713c089d88_0.png"
]

# Define the prompt
prompt = """You are a cybersecurity expert analyzing various images. You are tasked to classify the images into one of the following categories. Look for titles or any clues that can help with classification:
1. Financial Information (if it contains salaries, bonuses, or any other related terms)
2. Employee Information (if it contains date of birth, full name, or any other personal information)
3. Product Documentation (if it has images of platforms, and presents any documentation related aspects)
4. Chats (regular conversations or just message(s))

This is aimed at a 0-shot classification. Assign ONLY ONE CATEGORY TO THE FILE. I DONT NEED ANY REASONING, ONLY output the number afferent to the category.
"""

# Function to classify a single image
def classify_image(image_path):
    try:
        with open(image_path, "rb") as f:
            image_b64 = base64.b64encode(f.read()).decode("utf-8")

        response = requests.post(
            "http://localhost:11434/api/generate",
            json={
                "model": "qwen2.5vl:32b",
                "prompt": prompt,
                "images": [image_b64],
                "stream": False
            },
            timeout=1000
        )

        filename = os.path.basename(image_path)
        if response.ok:
            output_text = response.json()["response"].strip()
        else:
            output_text = f"Error: {response.text}"

        return filename, output_text

    except Exception as e:
        return os.path.basename(image_path), f"Exception: {e}"

# Track execution time
start_time = time.time()

# Run classification in parallel with progress bar
results = []
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = {executor.submit(classify_image, path): path for path in image_paths}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Classifying Images"):
        filename, result = future.result()
        results.append((filename, result))

# Output results
for filename, result in results:
    print(f"{filename}:\n{result}\n")

# Print total time taken
total_time = time.time() - start_time
print(f"Completed in {total_time:.2f} seconds.")

Classifying Images: 100%|██████████| 14/14 [03:38<00:00, 15.57s/it]

12756724-394c-4576-b373-7c53f1abbd94_39.png:
3

3348953d-66e9-4cac-8675-65bb5f2ef929_4_0.png:
1

12756724-394c-4576-b373-7c53f1abbd94_47.png:
3

3348953d-66e9-4cac-8675-65bb5f2ef929_5_0.png:
1

3348953d-66e9-4cac-8675-65bb5f2ef929_3_0.png:
1

3348953d-66e9-4cac-8675-65bb5f2ef929_2_0.png:
1

3348953d-66e9-4cac-8675-65bb5f2ef929_2_1.png:
1

3348953d-66e9-4cac-8675-65bb5f2ef929_1_0.png:
1

12756724-394c-4576-b373-7c53f1abbd94_9.png:
3

12756724-394c-4576-b373-7c53f1abbd94_49.png:
3

585875ff-f8c5-4a02-acd7-fef37dc9ff11_5.png:
3

645dfc97-3268-4e1d-920d-4138545456fa.png:
4

6e9aced1-df28-4e57-b7c8-641609ff4450.png:
4

6d7fc7b3-c892-4cb5-bd4b-a5713c089d88_0.png:
2

Completed in 218.12 seconds.


In [7]:
import base64
import requests
import os
import time
from tqdm import tqdm

# List of image file paths
image_paths = [
    # Financial Info Here (Employee Salary)
    "0/png/3348953d-66e9-4cac-8675-65bb5f2ef929_1_0.png",
    "0/png/3348953d-66e9-4cac-8675-65bb5f2ef929_2_0.png",
    "0/png/3348953d-66e9-4cac-8675-65bb5f2ef929_2_1.png",
    "0/png/3348953d-66e9-4cac-8675-65bb5f2ef929_3_0.png",
    "0/png/3348953d-66e9-4cac-8675-65bb5f2ef929_4_0.png",
    "0/png/3348953d-66e9-4cac-8675-65bb5f2ef929_5_0.png",

    # Product Documentation
    "0/png/12756724-394c-4576-b373-7c53f1abbd94_39.png",
    "0/png/12756724-394c-4576-b373-7c53f1abbd94_47.png",
    "0/png/12756724-394c-4576-b373-7c53f1abbd94_9.png",
    "0/png/12756724-394c-4576-b373-7c53f1abbd94_49.png",
    "0/png/585875ff-f8c5-4a02-acd7-fef37dc9ff11_5.png",

    # Chats
    "0/png/645dfc97-3268-4e1d-920d-4138545456fa.png",
    "0/png/6e9aced1-df28-4e57-b7c8-641609ff4450.png",

    # Employee Information
    "0/png/6d7fc7b3-c892-4cb5-bd4b-a5713c089d88_0.png"
]

# Define the prompt
prompt = """You are a cybersecurity expert analyzing various images. You are tasked to classify the images into one of the following categories. Look for titles or any clues that can help with classification:
1. Financial Information (if it contains salaries, bonuses, or any other related terms)
2. Employee Information (if it contains date of birth, full name, or any other personal information)
3. Product Documentation (if it has images of platforms, and presents any documentation related aspects)
4. Chats (regular conversations or just message(s))
"""

# Start timer
start_time = time.time()

# Process each image with progress bar
for image_path in tqdm(image_paths, desc="Classifying Images"):
    try:
        with open(image_path, "rb") as f:
            image_b64 = base64.b64encode(f.read()).decode("utf-8")

        response = requests.post(
            "http://localhost:11434/api/generate",
            json={
                "model": "qwen2.5vl:32b",
                "prompt": prompt,
                "images": [image_b64],
                "stream": False
            },
            timeout=90
        )

        filename = os.path.basename(image_path)
        if response.ok:
            output_text = response.json()["response"]
        else:
            output_text = f"Error: {response.text}"

    except Exception as e:
        filename = os.path.basename(image_path)
        output_text = f"Exception: {e}"

    print(f"{filename}:\n{output_text.strip()}\n")

# End timer
end_time = time.time()
print(f"Completed in {end_time - start_time:.2f} seconds.")

Classifying Images:   7%|▋         | 1/14 [00:16<03:35, 16.60s/it]

3348953d-66e9-4cac-8675-65bb5f2ef929_1_0.png:
2



Classifying Images:  14%|█▍        | 2/14 [00:33<03:18, 16.52s/it]

3348953d-66e9-4cac-8675-65bb5f2ef929_2_0.png:
2



Classifying Images:  21%|██▏       | 3/14 [00:50<03:04, 16.74s/it]

...

645dfc97-3268-4e1d-920d-4138545456fa.png:
4



Classifying Images:  93%|█████████▎| 13/14 [03:09<00:09,  9.83s/it]

6e9aced1-df28-4e57-b7c8-641609ff4450.png:
4



Classifying Images: 100%|██████████| 14/14 [03:26<00:00, 14.74s/it]

6d7fc7b3-c892-4cb5-bd4b-a5713c089d88_0.png:
2

Completed in 206.39 seconds.
